In [1]:
import math
import numpy as np
np.float = float
np.int   = int
body_imu = [1.72, -15.86, -0.08]
crotch_imu = [1.86, -14.88, 0.57]
# body_imu = [-1.10, -1.36, 0.90]
# crotch_imu = [-1.01, -0.62, -14.12]
body_imu_rad    = np.deg2rad(body_imu)
crotch_imu_rad  = np.deg2rad(crotch_imu)
# --- helper 함수들 ---
#회전 행렬 만들기
#x축으로 roll, y축으로 pitch, z축으로 yaw
def rpy_to_mat(roll, pitch, yaw): 
    
    Rx = np.array([[1, 0, 0],
                   [0, math.cos(roll), -math.sin(roll)],
                   [0, math.sin(roll),  math.cos(roll)]])
    Ry = np.array([[ math.cos(pitch), 0, math.sin(pitch)],
                   [0, 1, 0],
                   [-math.sin(pitch), 0, math.cos(pitch)]])
    Rz = np.array([[math.cos(yaw), -math.sin(yaw), 0],
                   [math.sin(yaw),  math.cos(yaw), 0],
                   [0, 0, 1]])
    return Rz @ Ry @ Rx


def axis_angle_to_mat(axis, angle):
    axis = np.array(axis) / np.linalg.norm(axis) #단위 벡터로 만들기
    x,y,z = axis #축을 x,y,z로 나누기
    c, s = math.cos(angle), math.sin(angle) #삼각함수 각도 구하기
    C = 1-c #1-cos 저장
    # Rodrigues 회전 공식 적용
    return np.array([
      [x*x*C + c,   x*y*C - z*s, x*z*C + y*s],
      [y*x*C + z*s, y*y*C + c,   y*z*C - x*s],
      [z*x*C - y*s, z*y*C + x*s, z*z*C + c  ]
    ])

In [2]:
import json
from urdfpy import URDF

ROOT_PATH = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001'
# 1. URDF와 data.json 파일 경로
URDF_PATH   = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/g1_inspire_hand_description/g1_29dof_rev_1_0_with_inspire_hand_FTP.urdf'    # URDF가 담긴 .txt 파일
DATA_PATH   = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/data.json'   # data.json 파일


finger_map = {'thumb':5, 'thumb2':4, 'index':3, 'middle':2, 'ring':1, 'pinky':0}

LEFTRIGHT = 'left'
WHICH_FINGER = 'index'
FRAME = 942

names = [LEFTRIGHT, 'shoulder', 'elbow', 'wrist', 'pitch', 'roll', 'yaw']   #얘는 list로 비교하면서 바꿨고 (아래 for i, joint in enumerate(joints) 부분에서)
handmap = {
                f'k{LEFTRIGHT.capitalize()}HandThumbRotation' : f'{LEFTRIGHT}_thumb_1_joint',
                f'k{LEFTRIGHT.capitalize()}HandThumbBend' : f'{LEFTRIGHT}_thumb_3_joint',
                f'k{LEFTRIGHT.capitalize()}HandIndex' : f'{LEFTRIGHT}_index_1_joint',
                f'k{LEFTRIGHT.capitalize()}HandMiddle' : f'{LEFTRIGHT}_middle_1_joint',
                f'k{LEFTRIGHT.capitalize()}HandRing' : f'{LEFTRIGHT}_ring_1_joint',
                f'k{LEFTRIGHT.capitalize()}HandPinky' : f'{LEFTRIGHT}_little_1_joint',
            }

# 2. URDF 로드
robot = URDF.load(URDF_PATH)

# 3. data.json 로드
with open(DATA_PATH, 'r') as f:
    data = json.load(f)


joints  = data['info']['joint_names'][f'{LEFTRIGHT}_arm']  # 7개 joint names list       여기부터

for i, joint in enumerate(joints):
    tmp_name = ""

    joint = joint[1:]
    for name in names:
        find_res = joint.lower().find(name)
        if find_res == -1:
            continue
        else:
            tmp_name = tmp_name + joint[:find_res+len(name)] + '_'
            joint = joint[find_res+len(name):]
    
    joints[i] = (tmp_name[:-1] + '_joint').lower()

# 4. idx=0의 left_arm 관절각(qpos)과 joint 이름 목록 가져오기
qpos    = data['data'][FRAME]['states'][f'{LEFTRIGHT}_arm']['qpos']                         #여기까지는 arm을 뽑는부분

hand_qpos  = data['data'][FRAME]['states'][f'{LEFTRIGHT}_hand']['qpos']                 #여기부터
hand_joints = data['info']['joint_names'][f'{LEFTRIGHT}_hand']
hand_joints = [handmap[x] for x in hand_joints]                                       #여기까지는 hand joints 정리해두는부분 이중에 한개만 뽑아서 넣을거임

# 5. joint name → angle 매핑
q_dict = {name: angle for name, angle in zip(joints, qpos)}

q_dict[hand_joints[finger_map[WHICH_FINGER]]] = hand_qpos[finger_map[WHICH_FINGER]]       #hand q_dict에 추가

T_world_base = np.eye(4)
T_world_base[:3,:3] = rpy_to_mat(*body_imu_rad)
T_world_base[:3,3] = [0,0,crotch_imu_rad[2]]
# 6. forward kinematics 계산  
# ee_link = f'{LEFTRIGHT}_wrist_yaw_link'     # wrist뽑을때 uncomment, hand 뽑을때 comment
ee_link = hand_joints[finger_map[WHICH_FINGER]][:-6]                    # hand 뽑을때 uncomment, wrist 뽑을때 comment
print(ee_link)
# link_fk()는 각 링크별로 월드 기준 4×4 변환행렬을 반환
fk = robot.link_fk(q_dict)
# for link_obj, T in fk.items():
#     print(link_obj.name)
# T_ee_world = fk[ee_link]   # (4,4) 행렬
ee_link_obj  = robot.link_map[ee_link]    # 이름→Link 객체 얻기
T_ee_world   = fk[ee_link_obj]                 # 올바르게 인덱싱!

# 7. 위치(translation) 추출
pos_ee_world = T_world_base@T_ee_world 
print('Left arm end-effector position in world frame:', pos_ee_world)


left_index_1
Left arm end-effector position in world frame: [[ 0.49976709  0.25524336  0.82769782  0.26168679]
 [ 0.55512993 -0.82791925 -0.07987792  0.18907581]
 [ 0.66487865  0.49940019 -0.55546002  0.30870893]
 [ 0.          0.          0.          1.        ]]


/home/scilab/anaconda3/envs/jy_tv/lib/python3.10/site-packages/urdfpy/urdf.py:2169: RuntimeWarning: invalid value encountered in divide
  value = value / np.linalg.norm(value)


In [3]:
# cam = robot['d435I_joint']
T_world_cam = np.eye(4)
T_world_cam[:3,:3] = rpy_to_mat(0, 0.8307767239493009, 0)
T_world_cam[:3, 3] = [0.0576235, 0.01753, 0.42987]   # :contentReference[oaicite:0]{index=0}


# 5) camera→left_wrist 변환
T_cam_ee = np.linalg.inv(T_world_cam) @ pos_ee_world

# 6) translation 성분이 바로 “카메라 기준” 손끝 좌표
pos_cam_ee = T_cam_ee[:3, 3]
print("Left wrist position in D435 camera frame:", pos_cam_ee)

Left wrist position in D435 camera frame: [0.2270724  0.17154581 0.06899243]


In [4]:
import cv2
import numpy as np

# 1) Intrinsics
width, height = 848, 480
fx = 426.7954
fy = 426.7954
# cx = width  / 2  # 424
# cy = height / 2  # 320
# fx=605.9278 
# fy=606.1917 
# cx = width  / 2  
# cy = height / 2  
cx = 414.0560
cy = 260.2649
# 2) pos_cam_ee: (X, Y, Z)  
#    X: 오른쪽(+), Y: 카메라 앞쪽(+), Z: 위쪽(+)
# print(pos_cam_ee)
# Xc =  pos_cam_ee[0]      
# Yc =  pos_cam_ee[1]      
# Zc =  pos_cam_ee[2]     
# Xc, Yc, Zc = T_cam_ee[:3,3]
X_urdf, Y_urdf, Z_urdf = pos_cam_ee

# 1) 축 재배열
Xc =  X_urdf
Yc = -Y_urdf
Zc = -Z_urdf
# 2) 투영
u = fx * (Yc / Xc) + cx
v = fy * (Zc / Xc) + cy

# 소수점→정수 픽셀 좌표
print(u,v)
pt = (int(round(u)), int(round(v)))
print(pt)
# 4) 이미지 읽고 표시  
img = cv2.imread("/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000942_color_0.jpg")
print(FRAME)
if img is None:
    raise RuntimeError("Unable to load image.")
cv2.circle(img, pt, 8, (255,0,0), thickness=-1)  # 빨간 점
cv2.putText(img, f"{pt}", (pt[0]+10, pt[1]),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)

# 5) 결과 저장 (Jupyter에서 imshow 대신)
cv2.imwrite(f"projected_pointindex2{FRAME}.png", img)
print("Saved to projected_point.png")


91.62596439076509 130.58973241586406
(92, 131)
942
Saved to projected_point.png


In [9]:

import numpy as np
import json
import cv2
from pathlib import Path
from urdfpy import URDF
from scipy.optimize import least_squares

# =========================================================
# 경로 / 설정
# =========================================================
BODY_URDF_PATH  = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/g1/g1_body29_hand14.urdf'
LEFT_HAND_URDF  = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/inspire_hand/inspire_hand_left.urdf'
RIGHT_HAND_URDF = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/inspire_hand/inspire_hand_right.urdf'  # 없으면 '' 또는 None
DATA_PATH       = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/data.json'
IMG_DIR         = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors'
IMG_FMT         = '{:06d}_color_0.jpg'

OUT_DIR         = './palm_tip_vis'
CLICKS_JSON     = './clicks_palm_tip.json'   # 클릭 저장/불러올 파일

# 사용할 프레임들
FRAMES_VIS   = [0,100,200,300,400,500,600,900,1300,1500,1600,942]

# ---- 클릭 수집 방식 ----
USE_GUI      = True    # GUI가 안되면 False
SAVE_CLICKS  = True    # 수집한 클릭을 JSON으로 저장
LOAD_CLICKS  = False   # 이전에 저장한 클릭을 불러와 재사용

# 카메라 intrinsics
fx, fy = 605.9278, 606.1917
cx, cy = 414.0560, 260.2649
K = np.array([[fx, 0, cx],
              [0, fy, cy],
              [0,  0,  1]], dtype=float)
dist = np.zeros(5)

# ===== 카메라 Extrinsics (camera ← world) : 이전 공동최적화 결과 =====
R_cw = np.array([
    [ 0.22643170, -0.96805671, -0.10767957],
    [-0.36084943,  0.01931217, -0.93242411],
    [ 0.90471895,  0.24998648, -0.34494983]
], dtype=float)
t_cw = np.array([-0.01571018, 0.35198288, 0.29489303], dtype=float)

# 손목 오프셋 (손목 로컬 좌표계) — 공동최적화 결과
dL = np.array([-0.01958220,  0.03467778, -0.00843096], dtype=float)   # left
dR = np.array([ 0.00669406, -0.00187255,  0.03843392], dtype=float)   # right

# 손끝 장착 미세보정 (회전/이동) — 필요 없으면 0
dRvec_L = np.zeros(3)
dT_L    = np.zeros(3)
dRvec_R = np.zeros(3)
dT_R    = np.zeros(3)

# 손바닥 중심 오프셋(손 베이스 프레임에서) — 기본 0
pPalm_L = np.array([0.0, 0.0, 0.0], dtype=float)
pPalm_R = np.array([0.0, 0.0, 0.0], dtype=float)

# 로봇 바디 IMU (deg→rad)
body_imu = np.deg2rad([1.72, -15.86, -0.08])  # roll, pitch, yaw

# 색상
COL_LW = (0,255,255)   # 왼손목
COL_RW = (255,255,0)   # 오른손목
COL_LT = (0,0,255)     # 왼손끝
COL_RT = (255,0,0)     # 오른손끝
COL_LP = (0,255,0)     # 왼손바닥
COL_RP = (255,0,255)   # 오른손바닥

# =========================================================
# 유틸
# =========================================================
def rpy_to_mat(r, p, y):
    Rx = np.array([[1,0,0],[0,np.cos(r),-np.sin(r)],[0,np.sin(r),np.cos(r)]])
    Ry = np.array([[np.cos(p),0,np.sin(p)],[0,1,0],[-np.sin(p),0,np.cos(p)]])
    Rz = np.array([[np.cos(y),-np.sin(y),0],[np.sin(y),np.cos(y),0],[0,0,1]])
    return Rz @ Ry @ Rx

def rodrigues_to_R(rvec):
    theta = np.linalg.norm(rvec)
    if theta < 1e-12:
        return np.eye(3)
    k = rvec / theta
    Kc = np.array([[0, -k[2], k[1]],
                   [k[2], 0, -k[0]],
                   [-k[1], k[0], 0]])
    return np.eye(3) + np.sin(theta)*Kc + (1-np.cos(theta))*(Kc@Kc)

def project_point_world(p_world):
    """world 3D → pixel(+카메라 좌표)"""
    p_cam = R_cw @ p_world + t_cw
    X, Y, Z = p_cam
    if Z <= 0:
        return None, p_cam
    u = fx * (X / Z) + cx
    v = fy * (Y / Z) + cy
    return (int(round(u)), int(round(v))), p_cam

# ---------- JSON 직렬화 유틸 ----------
def clicks_to_jsonable(clicks: dict) -> dict:
    """numpy를 모두 list/float/None으로 변환"""
    out = {}
    for fr, entry in clicks.items():
        e2 = {}
        for k, v in entry.items():   # LW/RW/LT/RT/LP/RP
            if v is None:
                e2[k] = None
            else:
                e2[k] = np.asarray(v, dtype=float).tolist()
        out[str(fr)] = e2
    return out

def clicks_from_json(raw: dict) -> dict:
    """JSON(dict of lists/None)을 numpy로 변환"""
    out = {}
    for fr, entry in raw.items():
        e2 = {}
        for k, v in entry.items():
            e2[k] = (np.array(v, dtype=float) if v is not None else None)
        out[str(fr)] = e2
    return out

# ---------- 클릭 도구 ----------
class Clicker:
    def __init__(self, scale=1.0):
        self.scale = scale
        self.pt = None
        self.win = None

    def _cb(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            self.pt = (x/self.scale, y/self.scale)
            disp = param
            cv2.circle(disp, (x,y), 6, (0,0,255), -1)
            cv2.imshow(self.win, disp)

    def click_or_none(self, img_path, label):
        if not USE_GUI:
            print(f'[skip {label}] GUI 비활성화')
            return None
        img = cv2.imread(img_path)
        if img is None:
            print(f'[skip {label}] image not found: {img_path}')
            return None
        h, w = img.shape[:2]
        disp = cv2.resize(img, (int(w*self.scale), int(h*self.scale)))
        self.pt = None
        self.win = f'click_{label}'
        try:
            cv2.imshow(self.win, disp)
            cv2.setMouseCallback(self.win, self._cb, disp)
            print(f'[ {label} ] {img_path} — 좌클릭(선택) / s 또는 ESC 건너뛰기')
            while True:
                k = cv2.waitKey(0) & 0xFF
                if k in (27, ord('s')):  # ESC or 's'
                    cv2.destroyWindow(self.win)
                    return None
                if self.pt is not None:
                    cv2.destroyWindow(self.win)
                    return np.array(self.pt, dtype=float)
        except cv2.error as e:
            print(f'[skip {label}] OpenCV GUI error: {e}')
            try: cv2.destroyWindow(self.win)
            except: pass
            return None

# =========================================================
# 데이터/URDF 로드 및 맵
# =========================================================
print('[load] URDF ...')
robot = URDF.load(BODY_URDF_PATH)
left_hand = URDF.load(LEFT_HAND_URDF)
right_hand = None
if RIGHT_HAND_URDF:
    try:
        right_hand = URDF.load(RIGHT_HAND_URDF)
    except Exception as e:
        print(f"[WARN] RIGHT_HAND_URDF 로드 실패: {e}. 오른손 요소 생략.")
        right_hand = None

print('[load] data.json ...')
with open(DATA_PATH, 'r') as f:
    data = json.load(f)

# 팔/손 매핑
l_arm_map = {
    'kLeftShoulderPitch':'left_shoulder_pitch_joint',
    'kLeftShoulderRoll' :'left_shoulder_roll_joint',
    'kLeftShoulderYaw'  :'left_shoulder_yaw_joint',
    'kLeftElbow'        :'left_elbow_joint',
    'kLeftWristRoll'    :'left_wrist_roll_joint',
    'kLeftWristPitch'   :'left_wrist_pitch_joint',
    'kLeftWristyaw'     :'left_wrist_yaw_joint',
}
r_arm_map = {
    'kRightShoulderPitch':'right_shoulder_pitch_joint',
    'kRightShoulderRoll' :'right_shoulder_roll_joint',
    'kRightShoulderYaw'  :'right_shoulder_yaw_joint',
    'kRightElbow'        :'right_elbow_joint',
    'kRightWristRoll'    :'right_wrist_roll_joint',
    'kRightWristPitch'   :'right_wrist_pitch_joint',
    'kRightWristYaw'     :'right_wrist_yaw_joint',
}
l_hand_map = {
    'kLeftHandPinky':'L_pinky_proximal_joint',
    'kLeftHandRing' :'L_ring_proximal_joint',
    'kLeftHandMiddle':'L_middle_proximal_joint',
    'kLeftHandIndex':'L_index_proximal_joint',
    'kLeftHandThumbBend':'L_thumb_proximal_pitch_joint',
    'kLeftHandThumbRotation':'L_thumb_proximal_yaw_joint',
}
# 오른손 키 이름은 실제 data['info']['joint_names']['right_hand'] 확인 후 맞춰주세요.
r_hand_map = {
    'kRightHandPinky':'R_pinky_proximal_joint',
    'kRightHandRing' :'R_ring_proximal_joint',
    'kRightHandMiddle':'R_middle_proximal_joint',
    'kRightHandIndex':'R_index_proximal_joint',
    'kRightHandThumbBend':'R_thumb_proximal_pitch_joint',
    'kRightHandThumbRotation':'R_thumb_proximal_yaw_joint',
}

# palm / tip 링크명 (URDF에 맞게 수정 가능)
L_PALM_LINK = 'L_hand_base_link'   # 좌손 바닥 링크
R_PALM_LINK = 'R_hand_base_link'   # 우손 바닥 링크 (파일 확인 필요)
L_TIP_LINK  = 'L_index_tip'
R_TIP_LINK  = 'R_index_tip'

def safe_q(names_list, qpos_list, mapping):
    q = {}
    for n, v in zip(names_list, qpos_list):
        if n in mapping:
            q[mapping[n]] = v
    return q

# =========================================================
# BODY URDF에서 wrist→hand 장착 변환 자동 추출
# =========================================================
def find_mount_transform(robot, wrist_link, prefer_side='left'):
    cand = []
    for j in robot.joints:
        if j.parent == wrist_link:
            child_low = j.child.lower()
            if ('hand' in child_low) or ('finger' in child_low):
                cand.append(j)
    if not cand:
        return None, None
    picked = cand[0]
    if len(cand) > 1:
        for j in cand:
            if prefer_side == 'left' and ('l_' in j.child.lower() or 'left' in j.child.lower()):
                picked = j; break
            if prefer_side == 'right' and ('r_' in j.child.lower() or 'right' in j.child.lower()):
                picked = j; break
    return picked, picked.origin  # origin은 4x4 동차변환

jL, T_wrist_to_hand_L = find_mount_transform(robot, 'left_wrist_yaw_link', 'left')
jR, T_wrist_to_hand_R = find_mount_transform(robot, 'right_wrist_yaw_link', 'right')

if T_wrist_to_hand_L is None:
    print('[WARN] left wrist->hand mount not found. Use Identity.')
    T_wrist_to_hand_L = np.eye(4)
else:
    print('[MOUNT-LEFT] parent:', jL.parent, 'child:', jL.child)

if T_wrist_to_hand_R is None:
    print('[WARN] right wrist->hand mount not found. Use Identity.')
    T_wrist_to_hand_R = np.eye(4)
else:
    print('[MOUNT-RIGHT] parent:', jR.parent, 'child:', jR.child)

# =========================================================
# FK
# =========================================================
def wrist_pose_world(frame, side='left'):
    if side == 'left':
        names = data['info']['joint_names']['left_arm']
        qpos  = data['data'][frame]['states']['left_arm']['qpos']
        amap  = l_arm_map
        wrist_link = 'left_wrist_yaw_link'
    else:
        names = data['info']['joint_names']['right_arm']
        qpos  = data['data'][frame]['states']['right_arm']['qpos']
        amap  = r_arm_map
        wrist_link = 'right_wrist_yaw_link'

    q = { amap[n]: v for n,v in zip(names, qpos) if n in amap }
    fk = robot.link_fk(q)

    T_world_base = np.eye(4)
    T_world_base[:3,:3] = rpy_to_mat(*body_imu)
    T_wrist = T_world_base @ fk[ robot.link_map[wrist_link] ]
    Rw = T_wrist[:3,:3]
    pw = T_wrist[:3,3]
    return Rw, pw

def left_index_tip_world(frame, d_off=dL, dRvec=dRvec_L, dT=dT_L):
    Rw, pw = wrist_pose_world(frame, 'left')
    T_off = np.eye(4); T_off[:3,3] = d_off
    R_delta = rodrigues_to_R(dRvec)
    T_mount = T_wrist_to_hand_L.copy()
    T_mount[:3,:3] = R_delta @ T_mount[:3,:3]
    T_mount[:3,3] += dT

    names = data['info']['joint_names']['left_hand']
    qpos  = data['data'][frame]['states']['left_hand']['qpos']
    qh = safe_q(names, qpos, l_hand_map)
    fk_h = left_hand.link_fk(qh)
    if L_TIP_LINK not in left_hand.link_map:
        return None
    T_hand_tip = fk_h[left_hand.link_map[L_TIP_LINK]]

    T_world_wrist = np.eye(4); T_world_wrist[:3,:3]=Rw; T_world_wrist[:3,3]=pw
    T_world_tip = T_world_wrist @ T_off @ T_mount @ T_hand_tip
    return T_world_tip[:3,3]

def right_index_tip_world(frame, d_off=dR, dRvec=dRvec_R, dT=dT_R):
    if right_hand is None or (R_TIP_LINK not in getattr(right_hand, 'link_map', {})):
        return None
    Rw, pw = wrist_pose_world(frame, 'right')
    T_off = np.eye(4); T_off[:3,3] = d_off
    R_delta = rodrigues_to_R(dRvec)
    T_mount = T_wrist_to_hand_R.copy()
    T_mount[:3,:3] = R_delta @ T_mount[:3,:3]
    T_mount[:3,3] += dT

    names = data['info']['joint_names']['right_hand']
    qpos  = data['data'][frame]['states']['right_hand']['qpos']
    qh = safe_q(names, qpos, r_hand_map)
    fk_h = right_hand.link_fk(qh)
    T_hand_tip = fk_h[right_hand.link_map[R_TIP_LINK]]

    T_world_wrist = np.eye(4); T_world_wrist[:3,:3]=Rw; T_world_wrist[:3,3]=pw
    T_world_tip = T_world_wrist @ T_off @ T_mount @ T_hand_tip
    return T_world_tip[:3,3]

# ---------- Palm (base) ----------
def left_palm_world(frame, d_off=dL, palm_off=pPalm_L):
    """손목 → (오프셋) → wrist->hand mount → hand base → palm center."""
    Rw, pw = wrist_pose_world(frame, 'left')
    T_world_wrist = np.eye(4); T_world_wrist[:3,:3]=Rw; T_world_wrist[:3,3]=pw
    T_off = np.eye(4); T_off[:3,3] = d_off
    T_mount = T_wrist_to_hand_L
    T_palm = np.eye(4); T_palm[:3,3] = palm_off
    T_world_palm = T_world_wrist @ T_off @ T_mount @ T_palm
    return T_world_palm[:3,3]

def right_palm_world(frame, d_off=dR, palm_off=pPalm_R):
    if right_hand is None:
        return None
    Rw, pw = wrist_pose_world(frame, 'right')
    T_world_wrist = np.eye(4); T_world_wrist[:3,:3]=Rw; T_world_wrist[:3,3]=pw
    T_off = np.eye(4); T_off[:3,3] = d_off
    T_mount = T_wrist_to_hand_R
    T_palm = np.eye(4); T_palm[:3,3] = palm_off
    T_world_palm = T_world_wrist @ T_off @ T_mount @ T_palm
    return T_world_palm[:3,3]

# =========================================================
# 클릭 수집/불러오기 (손목/손끝/손바닥)
# =========================================================
# 항목 키: LW, RW, LT, RT, LP, RP
def collect_clicks(frames):
    if LOAD_CLICKS and Path(CLICKS_JSON).exists():
        print(f'[load clicks] {CLICKS_JSON}')
        with open(CLICKS_JSON, 'r') as f:
            raw = json.load(f)
        return clicks_from_json(raw)

    clicker = Clicker(scale=1.0)
    clicks = {}
    for fr in frames:
        entry = {'LW':None,'RW':None,'LT':None,'RT':None,'LP':None,'RP':None}
        img_path = str(Path(IMG_DIR) / IMG_FMT.format(fr))
        print(f'\n[frame {fr}] 클릭 수집 시작')
        entry['LW'] = clicker.click_or_none(img_path, f'{fr}-LW(wrist-left)')
        entry['RW'] = clicker.click_or_none(img_path, f'{fr}-RW(wrist-right)')
        entry['LT'] = clicker.click_or_none(img_path, f'{fr}-LT(left tip)')
        entry['RT'] = clicker.click_or_none(img_path, f'{fr}-RT(right tip)')
        entry['LP'] = clicker.click_or_none(img_path, f'{fr}-LP(left palm)')
        entry['RP'] = clicker.click_or_none(img_path, f'{fr}-RP(right palm)')

        # 내부에는 numpy로 저장 (계산 편의)
        for k in entry:
            if entry[k] is not None:
                entry[k] = np.asarray(entry[k], dtype=float)

        clicks[str(fr)] = entry

    if SAVE_CLICKS:
        with open(CLICKS_JSON, 'w', encoding='utf-8') as f:
            json.dump(clicks_to_jsonable(clicks), f, indent=2, ensure_ascii=False)
        print(f'[save clicks] {CLICKS_JSON}')
    return clicks

# =========================================================
# (선택) palm 오프셋만 간단 보정
# =========================================================
def refine_palm_offset(side, frames, clicks_dict):
    """손바닥 중심 pPalm_* (hand base frame) 3D만 추정"""
    if side == 'left':
        p0 = pPalm_L.copy()
        def palm_world(fr, p):
            return left_palm_world(fr, d_off=dL, palm_off=p)
        key = 'LP'
    else:
        if right_hand is None:
            print('[refine palm] right hand 없음. skip.')
            return np.zeros(3)
        p0 = pPalm_R.copy()
        def palm_world(fr, p):
            return right_palm_world(fr, d_off=dR, palm_off=p)
        key = 'RP'

    obs_list = []
    for fr in frames:
        ent = clicks_dict.get(str(fr), {})
        uv = ent.get(key, None)
        if uv is None:
            continue
        obs_list.append((fr, np.array(uv, dtype=float)))

    if len(obs_list) < 3:
        print(f'[refine palm {side}] 관측 {len(obs_list)} < 3  skip.')
        return np.zeros(3)

    def resid(p):
        res = []
        for fr, uv_gt in obs_list:
            pw = palm_world(fr, p)
            uv, pc = project_point_world(pw)
            if uv is None:
                res += [1000.0, 1000.0]
            else:
                res += [uv[0]-uv_gt[0], uv[1]-uv_gt[1]]
        return np.array(res, dtype=float)

    r = least_squares(resid, p0, verbose=1, ftol=1e-9, xtol=1e-9, gtol=1e-9, max_nfev=200,
                      loss='soft_l1', f_scale=3.0)
    print(f'[refine palm {side}] status:', r.status, r.message, ' cost:', r.cost)
    res_err = resid(r.x).reshape(-1,2)
    e2 = np.linalg.norm(res_err, axis=1)
    print(f'[refine palm {side}] reproj mean/std(px):', e2.mean(), e2.std())
    return r.x - p0   # delta 반환

# =========================================================
# 메인: 시각화
# =========================================================
def main():
    Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

    clicks = collect_clicks(FRAMES_VIS)

    # (선택) palm 오프셋만 간단 보정 원하면 주석 해제
    # dPalm_L = refine_palm_offset('left',  FRAMES_VIS, clicks)
    # dPalm_R = refine_palm_offset('right', FRAMES_VIS, clicks)
    # global pPalm_L, pPalm_R
    # pPalm_L = pPalm_L + dPalm_L
    # pPalm_R = pPalm_R + dPalm_R

    for fr in FRAMES_VIS:
        img_path = str(Path(IMG_DIR) / IMG_FMT.format(fr))
        img = cv2.imread(img_path)
        if img is None:
            print('[WARN] image not found:', img_path)
            continue

        vis = img.copy()

        # wrist (오프셋 포함)
        RwL, pwL = wrist_pose_world(fr, 'left')
        RwR, pwR = wrist_pose_world(fr, 'right')
        p_wL = (RwL @ dL) + pwL
        p_wR = (RwR @ dR) + pwR
        uv_wL, _ = project_point_world(p_wL)
        uv_wR, _ = project_point_world(p_wR)

        # tip
        # p_tL = left_index_tip_world(fr)
        # p_tR = right_index_tip_world(fr)
        # uv_tL, _ = project_point_world(p_tL) if p_tL is not None else (None, None)
        # uv_tR, _ = project_point_world(p_tR) if p_tR is not None else (None, None)

        # palm
        p_pL = left_palm_world(fr, palm_off=pPalm_L)
        p_pR = right_palm_world(fr, palm_off=pPalm_R)
        uv_pL, _ = project_point_world(p_pL) if p_pL is not None else (None, None)
        uv_pR, _ = project_point_world(p_pR) if p_pR is not None else (None, None)

        # draw
        if uv_wL is not None:
            cv2.circle(vis, uv_wL, 8, COL_LW, -1); cv2.putText(vis, 'LW', (uv_wL[0]+6, uv_wL[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, COL_LW, 2)
        if uv_wR is not None:
            cv2.circle(vis, uv_wR, 8, COL_RW, -1); cv2.putText(vis, 'RW', (uv_wR[0]+6, uv_wR[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, COL_RW, 2)
        if uv_tL is not None:
            cv2.circle(vis, uv_tL, 7, COL_LT, -1); cv2.putText(vis, 'L_tip', (uv_tL[0]+6, uv_tL[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, COL_LT, 2)
        if uv_tR is not None:
            cv2.circle(vis, uv_tR, 7, COL_RT, -1); cv2.putText(vis, 'R_tip', (uv_tR[0]+6, uv_tR[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, COL_RT, 2)
        if uv_pL is not None:
            cv2.circle(vis, uv_pL, 7, COL_LP, -1); cv2.putText(vis, 'L_palm', (uv_pL[0]+6, uv_pL[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, COL_LP, 2)
        if uv_pR is not None:
            cv2.circle(vis, uv_pR, 7, COL_RP, -1); cv2.putText(vis, 'R_palm', (uv_pR[0]+6, uv_pR[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, COL_RP, 2)

        cv2.putText(vis, f'frame {fr}', (20,40), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0,255,0), 2)

        out_path = str(Path(OUT_DIR) / f'vis_{fr}.png')
        cv2.imwrite(out_path, vis)
        print('Saved:', out_path)

    print('\n완료!')
    print('- palm이 크게 어긋나면 pPalm_L/R을 약간 조정하고 다시 확인해봐.')
    print('- tip이 어긋나면 dRvec/dT (장착 회전/이동 보정)도 조정 가능.')
    print('- GUI가 안 뜨면 USE_GUI=False로 두고, CLICKS_JSON에 수동으로 픽셀 좌표를 채워 넣어도 돼.')

if __name__ == '__main__':
    main()


[load] URDF ...
[load] data.json ...
[MOUNT-LEFT] parent: left_wrist_yaw_link child: left_hand_palm_link
[MOUNT-RIGHT] parent: right_wrist_yaw_link child: right_hand_palm_link

[frame 0] 클릭 수집 시작
[ 0-LW(wrist-left) ] /home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000000_color_0.jpg — 좌클릭(선택) / s 또는 ESC 건너뛰기
[ 0-RW(wrist-right) ] /home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000000_color_0.jpg — 좌클릭(선택) / s 또는 ESC 건너뛰기
[ 0-LT(left tip) ] /home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000000_color_0.jpg — 좌클릭(선택) / s 또는 ESC 건너뛰기
[ 0-RT(right tip) ] /home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000000_color_0.jpg — 좌클릭(선택) / s 또는 ESC 건너뛰기
[ 0-LP(left palm) ] /home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000000_color_0.jpg — 좌클릭(선택) / s

In [14]:
# -*- coding: utf-8 -*-
"""
Wrist orientation → palm direction 기반으로 픽셀/3D 위치를 추정하는 전체 스크립트.
- 카메라 extrinsics, 손목 오프셋(dL/dR), 손끝/손바닥 투영
- 손목 픽셀에서 손바닥 방향으로 Dpx만큼 이동한 픽셀 추정
- 위 픽셀 이동량을 실제 3D 이동으로 환산해 palm 3D 점도 추정

필요 패키지: urdfpy, numpy, opencv-python, scipy
"""

import numpy as np
import cv2
from pathlib import Path
from urdfpy import URDF

# =========================================================
# 경로 / 설정
# =========================================================
BODY_URDF_PATH  = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/g1/g1_body29_hand14.urdf'
LEFT_HAND_URDF  = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/inspire_hand/inspire_hand_left.urdf'
RIGHT_HAND_URDF = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/inspire_hand/inspire_hand_right.urdf'
DATA_PATH       = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/data.json'
IMG_DIR         = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors'
IMG_FMT         = '{:06d}_color_0.jpg'

OUT_DIR         = './wrist_palm_dir_vis'
FRAMES_VIS      = [0,100,200,300,400,500,600,900,1300,1500,1600,942]

# 카메라 intrinsics
fx, fy = 605.9278, 606.1917
cx, cy = 414.0560, 260.2649

# ===== 카메라 Extrinsics (camera ← world) : 이전 공동최적화 결과 =====
R_cw = np.array([
    [ 0.22643170, -0.96805671, -0.10767957],
    [-0.36084943,  0.01931217, -0.93242411],
    [ 0.90471895,  0.24998648, -0.34494983]
], dtype=float)
t_cw = np.array([-0.01571018, 0.35198288, 0.29489303], dtype=float)

# 손목 오프셋 (손목 로컬 좌표계) — 공동최적화 결과
dL = np.array([-0.01958220,  0.03467778, -0.00843096], dtype=float)
dR = np.array([ 0.00669406, -0.00187255,  0.03843392], dtype=float)

# 손끝 장착 보정(현재 0)
dRvec_L = np.zeros(3); dT_L = np.zeros(3)
dRvec_R = np.zeros(3); dT_R = np.zeros(3)

# 손바닥 중심 오프셋(핸드 base 프레임) — 필요시 조정
pPalm_L = np.array([0.0, 0.0, 0.0], dtype=float)
pPalm_R = np.array([0.0, 0.0, 0.0], dtype=float)

# 로봇 바디 IMU (deg→rad)
body_imu = np.deg2rad([1.72, -15.86, -0.08])

# 손바닥 법선 방향(핸드 base 프레임). URDF 확인 후 부호가 반대면 [0,0,-1]로 바꿔.
N_HAND_AXIS = np.array([0, 0, 1.0], dtype=float)
# 픽셀-미터 스케일 측정을 위한 작은 이동(m)
S_TEST = 0.01  # 1cm
# 손목 픽셀에서 이만큼 이동시켜 palm 픽셀을 그려봄
DPIX_L = 20.0
DPIX_R = 20.0

# 색상
COL_LW = (0,255,255)
COL_RW = (255,255,0)
COL_LT = (0,0,255)
COL_RT = (255,0,0)
COL_LP = (0,255,0)
COL_RP = (255,0,255)
COL_PX_DIR = (0,200,0)

# =========================================================
# 유틸
# =========================================================
def rpy_to_mat(r, p, y):
    Rx = np.array([[1,0,0],[0,np.cos(r),-np.sin(r)],[0,np.sin(r),np.cos(r)]])
    Ry = np.array([[np.cos(p),0,np.sin(p)],[0,1,0],[-np.sin(p),0,np.cos(p)]])
    Rz = np.array([[np.cos(y),-np.sin(y),0],[np.sin(y),np.cos(y),0],[0,0,1]])
    return Rz @ Ry @ Rx

def rodrigues_to_R(rvec):
    theta = np.linalg.norm(rvec)
    if theta < 1e-12:
        return np.eye(3)
    k = rvec / theta
    Kc = np.array([[0, -k[2], k[1]],
                   [k[2], 0, -k[0]],
                   [-k[1], k[0], 0]])
    return np.eye(3) + np.sin(theta)*Kc + (1-np.cos(theta))*(Kc@Kc)

def project_point_world(p_world):
    p_cam = R_cw @ p_world + t_cw
    X, Y, Z = p_cam
    if Z <= 0:
        return None, p_cam
    u = fx * (X / Z) + cx
    v = fy * (Y / Z) + cy
    return (int(round(u)), int(round(v))), p_cam

# =========================================================
# 로드 및 맵
# =========================================================
robot = URDF.load(BODY_URDF_PATH)
left_hand = URDF.load(LEFT_HAND_URDF)
try:
    right_hand = URDF.load(RIGHT_HAND_URDF)
except Exception:
    right_hand = None

import json
with open(DATA_PATH, 'r') as f:
    data = json.load(f)

l_arm_map = {
    'kLeftShoulderPitch':'left_shoulder_pitch_joint',
    'kLeftShoulderRoll' :'left_shoulder_roll_joint',
    'kLeftShoulderYaw'  :'left_shoulder_yaw_joint',
    'kLeftElbow'        :'left_elbow_joint',
    'kLeftWristRoll'    :'left_wrist_roll_joint',
    'kLeftWristPitch'   :'left_wrist_pitch_joint',
    'kLeftWristyaw'     :'left_wrist_yaw_joint',
}
r_arm_map = {
    'kRightShoulderPitch':'right_shoulder_pitch_joint',
    'kRightShoulderRoll' :'right_shoulder_roll_joint',
    'kRightShoulderYaw'  :'right_shoulder_yaw_joint',
    'kRightElbow'        :'right_elbow_joint',
    'kRightWristRoll'    :'right_wrist_roll_joint',
    'kRightWristPitch'   :'right_wrist_pitch_joint',
    'kRightWristYaw'     :'right_wrist_yaw_joint',
}

l_hand_map = {
    'kLeftHandPinky':'L_pinky_proximal_joint',
    'kLeftHandRing' :'L_ring_proximal_joint',
    'kLeftHandMiddle':'L_middle_proximal_joint',
    'kLeftHandIndex':'L_index_proximal_joint',
    'kLeftHandThumbBend':'L_thumb_proximal_pitch_joint',
    'kLeftHandThumbRotation':'L_thumb_proximal_yaw_joint',
}
r_hand_map = {
    'kRightHandPinky':'R_pinky_proximal_joint',
    'kRightHandRing' :'R_ring_proximal_joint',
    'kRightHandMiddle':'R_middle_proximal_joint',
    'kRightHandIndex':'R_index_proximal_joint',
    'kRightHandThumbBend':'R_thumb_proximal_pitch_joint',
    'kRightHandThumbRotation':'R_thumb_proximal_yaw_joint',
}

L_TIP_LINK = 'L_index_tip'
R_TIP_LINK = 'R_index_tip'

# =========================================================
# wrist→hand mount 추출
# =========================================================
def find_mount_transform(robot, wrist_link, prefer_side='left'):
    cand = []
    for j in robot.joints:
        if j.parent == wrist_link:
            child_low = j.child.lower()
            if ('hand' in child_low) or ('finger' in child_low):
                cand.append(j)
    if not cand:
        return None, None
    picked = cand[0]
    if len(cand) > 1:
        for j in cand:
            if prefer_side == 'left' and ('l_' in j.child.lower() or 'left' in j.child.lower()):
                picked = j; break
            if prefer_side == 'right' and ('r_' in j.child.lower() or 'right' in j.child.lower()):
                picked = j; break
    return picked, picked.origin

jL, T_wrist_to_hand_L = find_mount_transform(robot, 'left_wrist_yaw_link', 'left')
jR, T_wrist_to_hand_R = find_mount_transform(robot, 'right_wrist_yaw_link', 'right')
if T_wrist_to_hand_L is None: T_wrist_to_hand_L = np.eye(4)
if T_wrist_to_hand_R is None: T_wrist_to_hand_R = np.eye(4)


def safe_q(names_list, qpos_list, mapping):
    q = {}
    for n, v in zip(names_list, qpos_list):
        if n in mapping:
            q[mapping[n]] = v
    return q

def wrist_pose_world(frame, side='left'):
    if side == 'left':
        names = data['info']['joint_names']['left_arm']
        qpos  = data['data'][frame]['states']['left_arm']['qpos']
        amap  = l_arm_map
        wrist_link = 'left_wrist_yaw_link'
    else:
        names = data['info']['joint_names']['right_arm']
        qpos  = data['data'][frame]['states']['right_arm']['qpos']
        amap  = r_arm_map
        wrist_link = 'right_wrist_yaw_link'

    q = { amap[n]: v for n,v in zip(names, qpos) if n in amap }
    fk = robot.link_fk(q)
    T_world_base = np.eye(4)
    T_world_base[:3,:3] = rpy_to_mat(*body_imu)
    T_wrist = T_world_base @ fk[ robot.link_map[wrist_link] ]
    Rw = T_wrist[:3,:3]
    pw = T_wrist[:3,3]
    return Rw, pw

def left_index_tip_world(frame, d_off=dL, dRvec=dRvec_L, dT=dT_L):
    Rw, pw = wrist_pose_world(frame, 'left')
    T_off = np.eye(4); T_off[:3,3] = d_off
    R_delta = rodrigues_to_R(dRvec)
    T_mount = T_wrist_to_hand_L.copy()
    T_mount[:3,:3] = R_delta @ T_mount[:3,:3]
    T_mount[:3,3] += dT
    names = data['info']['joint_names']['left_hand']
    qpos  = data['data'][frame]['states']['left_hand']['qpos']
    qh = safe_q(names, qpos, l_hand_map)
    fk_h = left_hand.link_fk(qh)
    if L_TIP_LINK not in left_hand.link_map:
        return None
    T_hand_tip = fk_h[left_hand.link_map[L_TIP_LINK]]
    T_world_wrist = np.eye(4); T_world_wrist[:3,:3]=Rw; T_world_wrist[:3,3]=pw
    T_world_tip = T_world_wrist @ T_off @ T_mount @ T_hand_tip
    return T_world_tip[:3,3]

def right_index_tip_world(frame, d_off=dR, dRvec=dRvec_R, dT=dT_R):
    if (right_hand is None) or (R_TIP_LINK not in getattr(right_hand, 'link_map', {})):
        return None
    Rw, pw = wrist_pose_world(frame, 'right')
    T_off = np.eye(4); T_off[:3,3] = d_off
    R_delta = rodrigues_to_R(dRvec)
    T_mount = T_wrist_to_hand_R.copy()
    T_mount[:3,:3] = R_delta @ T_mount[:3,:3]
    T_mount[:3,3] += dT
    names = data['info']['joint_names']['right_hand']
    qpos  = data['data'][frame]['states']['right_hand']['qpos']
    qh = safe_q(names, qpos, r_hand_map)
    fk_h = right_hand.link_fk(qh)
    T_hand_tip = fk_h[right_hand.link_map[R_TIP_LINK]]
    T_world_wrist = np.eye(4); T_world_wrist[:3,:3]=Rw; T_world_wrist[:3,3]=pw
    T_world_tip = T_world_wrist @ T_off @ T_mount @ T_hand_tip
    return T_world_tip[:3,3]

# palm = hand base + palm_off

def left_palm_world(frame, d_off=dL, palm_off=pPalm_L):
    Rw, pw = wrist_pose_world(frame, 'left')
    T_world_wrist = np.eye(4); T_world_wrist[:3,:3]=Rw; T_world_wrist[:3,3]=pw
    T_off = np.eye(4); T_off[:3,3] = d_off
    T_mount = T_wrist_to_hand_L
    T_palm = np.eye(4); T_palm[:3,3] = palm_off
    T_world_palm = T_world_wrist @ T_off @ T_mount @ T_palm
    return T_world_palm[:3,3]

def right_palm_world(frame, d_off=dR, palm_off=pPalm_R):
    if right_hand is None:
        return None
    Rw, pw = wrist_pose_world(frame, 'right')
    T_world_wrist = np.eye(4); T_world_wrist[:3,:3]=Rw; T_world_wrist[:3,3]=pw
    T_off = np.eye(4); T_off[:3,3] = d_off
    T_mount = T_wrist_to_hand_R
    T_palm = np.eye(4); T_palm[:3,3] = palm_off
    T_world_palm = T_world_wrist @ T_off @ T_mount @ T_palm
    return T_world_palm[:3,3]


def wrist_world_point(side, frame, use_offset=True):
    Rw, pw = wrist_pose_world(frame, side)
    if not use_offset:
        return pw
    if side == 'left':
        return (Rw @ dL) + pw
    else:
        return (Rw @ dR) + pw

def hand_mount_R(side):
    return T_wrist_to_hand_L[:3,:3] if side=='left' else T_wrist_to_hand_R[:3,:3]

def palm_dir_world(side, frame, n_hand=N_HAND_AXIS):
    Rw, _ = wrist_pose_world(frame, side)
    Rm = hand_mount_R(side)
    v = Rw @ (Rm @ n_hand)
    n = np.linalg.norm(v)
    if n < 1e-9:
        return np.array([0,0,1.0])
    return v / n

def pixel_dir_from_3d(side, frame, p_ref_world, s_test=S_TEST):
    v_world = palm_dir_world(side, frame)
    p2_world = p_ref_world + s_test * v_world
    uv1, _ = project_point_world(p_ref_world)
    uv2, _ = project_point_world(p2_world)
    if (uv1 is None) or (uv2 is None):
        return None, None, v_world
    duv = np.array([uv2[0]-uv1[0], uv2[1]-uv1[1]], dtype=float)
    scale_per_m = (np.linalg.norm(duv) / s_test) if s_test > 0 else None
    return duv, scale_per_m, v_world

def estimate_palm_pixel_by_wrist(side, frame, Dpx=20.0, use_wrist_offset=True):
    p_ref = wrist_world_point(side, frame, use_offset=use_wrist_offset)
    uv_ref, _ = project_point_world(p_ref)
    if uv_ref is None:
        return None
    duv, scale_per_m, _ = pixel_dir_from_3d(side, frame, p_ref)
    if duv is None:
        return None
    n = np.linalg.norm(duv)
    if n < 1e-9:
        return None
    dir_px = duv / n
    return (int(round(uv_ref[0] + Dpx * dir_px[0])),
            int(round(uv_ref[1] + Dpx * dir_px[1])))

def estimate_palm_3d_by_pixel_offset(side, frame, Dpx=20.0, use_wrist_offset=True):
    p_ref = wrist_world_point(side, frame, use_offset=use_wrist_offset)
    uv_ref, _ = project_point_world(p_ref)
    if uv_ref is None:
        return None, None
    duv, scale_per_m, v_world = pixel_dir_from_3d(side, frame, p_ref)
    if duv is None or scale_per_m is None or scale_per_m < 1e-6:
        return None, None
    s = Dpx / scale_per_m
    p_est = p_ref + s * v_world
    uv_est, _ = project_point_world(p_est)
    return uv_est, p_est

# =========================================================
# 시각화
# =========================================================

def main():
    Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

    for fr in FRAMES_VIS:
        img_path = str(Path(IMG_DIR) / IMG_FMT.format(fr))
        img = cv2.imread(img_path)
        if img is None:
            print('[WARN] image not found:', img_path)
            continue
        vis = img.copy()

        # 손목 (오프셋 적용 위치)
        RwL, pwL = wrist_pose_world(fr, 'left');  p_wL = (RwL @ dL) + pwL
        RwR, pwR = wrist_pose_world(fr, 'right'); p_wR = (RwR @ dR) + pwR
        uv_wL, _ = project_point_world(p_wL)
        uv_wR, _ = project_point_world(p_wR)

        if uv_wL is not None:
            cv2.circle(vis, uv_wL, 8, COL_LW, -1); cv2.putText(vis, 'LW', (uv_wL[0]+6, uv_wL[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, COL_LW, 2)
        if uv_wR is not None:
            cv2.circle(vis, uv_wR, 8, COL_RW, -1); cv2.putText(vis, 'RW', (uv_wR[0]+6, uv_wR[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, COL_RW, 2)

        # URDF palm (hand base + pPalm)
        p_pL = left_palm_world(fr, palm_off=pPalm_L)
        p_pR = right_palm_world(fr, palm_off=pPalm_R)
        uv_pL, _ = project_point_world(p_pL) if p_pL is not None else (None, None)
        uv_pR, _ = project_point_world(p_pR) if p_pR is not None else (None, None)
        if uv_pL is not None:
            cv2.circle(vis, uv_pL, 7, COL_LP, -1); cv2.putText(vis, 'L_palm(URDF)', (uv_pL[0]+6, uv_pL[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.6, COL_LP, 2)
        if uv_pR is not None:
            cv2.circle(vis, uv_pR, 7, COL_RP, -1); cv2.putText(vis, 'R_palm(URDF)', (uv_pR[0]+6, uv_pR[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.6, COL_RP, 2)

        # Wrist→palm 방향 기반 픽셀/3D 추정
        uv_palmL_px = estimate_palm_pixel_by_wrist('left', fr, Dpx=DPIX_L, use_wrist_offset=True)
        uv_palmR_px = estimate_palm_pixel_by_wrist('right', fr, Dpx=DPIX_R, use_wrist_offset=True)
        if uv_wL is not None and uv_palmL_px is not None:
            cv2.arrowedLine(vis, uv_wL, uv_palmL_px, COL_PX_DIR, 2, tipLength=0.25)
            cv2.circle(vis, uv_palmL_px, 6, COL_PX_DIR, -1)
            cv2.putText(vis, f'L_px+{int(DPIX_L)}', (uv_palmL_px[0]+6, uv_palmL_px[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.6, COL_PX_DIR, 2)
        if uv_wR is not None and uv_palmR_px is not None:
            cv2.arrowedLine(vis, uv_wR, uv_palmR_px, COL_PX_DIR, 2, tipLength=0.25)
            cv2.circle(vis, uv_palmR_px, 6, COL_PX_DIR, -1)
            cv2.putText(vis, f'R_px+{int(DPIX_R)}', (uv_palmR_px[0]+6, uv_palmR_px[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.6, COL_PX_DIR, 2)

        # 3D 환산 버전 (픽셀 Dpx → 실제 m 이동)도 함께 그리기
        uv_palmL_3d, p_palmL_3d = estimate_palm_3d_by_pixel_offset('left', fr, Dpx=DPIX_L, use_wrist_offset=True)
        uv_palmR_3d, p_palmR_3d = estimate_palm_3d_by_pixel_offset('right', fr, Dpx=DPIX_R, use_wrist_offset=True)
        if uv_palmL_3d is not None:
            cv2.circle(vis, uv_palmL_3d, 6, (0,128,0), -1)
            cv2.putText(vis, 'L_palm(3D)', (uv_palmL_3d[0]+6, uv_palmL_3d[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,128,0), 2)
        if uv_palmR_3d is not None:
            cv2.circle(vis, uv_palmR_3d, 6, (0,128,0), -1)
            cv2.putText(vis, 'R_palm(3D)', (uv_palmR_3d[0]+6, uv_palmR_3d[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,128,0), 2)

        cv2.putText(vis, f'frame {fr}', (20,40), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0,255,0), 2)
        out_path = str(Path(OUT_DIR) / f'vis_{fr}.png')
        cv2.imwrite(out_path, vis)
        print('Saved:', out_path)

if __name__ == '__main__':
    main()


Saved: wrist_palm_dir_vis/vis_0.png
Saved: wrist_palm_dir_vis/vis_100.png
Saved: wrist_palm_dir_vis/vis_200.png
Saved: wrist_palm_dir_vis/vis_300.png
Saved: wrist_palm_dir_vis/vis_400.png
Saved: wrist_palm_dir_vis/vis_500.png
Saved: wrist_palm_dir_vis/vis_600.png
Saved: wrist_palm_dir_vis/vis_900.png
Saved: wrist_palm_dir_vis/vis_1300.png
Saved: wrist_palm_dir_vis/vis_1500.png
Saved: wrist_palm_dir_vis/vis_1600.png
Saved: wrist_palm_dir_vis/vis_942.png


In [16]:
# -*- coding: utf-8 -*-
"""
Wrist orientation → palm direction 기반으로 픽셀/3D 위치를 추정하는 전체 스크립트.
- 카메라 extrinsics, 손목 오프셋(dL/dR), 손끝/손바닥 투영
- 손목 픽셀에서 손바닥 방향으로 Dpx만큼 이동한 픽셀 추정
- 위 픽셀 이동량을 실제 3D 이동으로 환산해 palm 3D 점도 추정
- URDF palm 투영점과 비교해 방향 부호(±)를 자동 선택

필요 패키지: urdfpy, numpy, opencv-python, scipy
"""

import numpy as np
import cv2
from pathlib import Path
from urdfpy import URDF
import json

# =========================================================
# 경로 / 설정
# =========================================================
BODY_URDF_PATH  = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/g1/g1_body29_hand14.urdf'
LEFT_HAND_URDF  = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/inspire_hand/inspire_hand_left.urdf'
RIGHT_HAND_URDF = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/inspire_hand/inspire_hand_right.urdf'
DATA_PATH       = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/data.json'
IMG_DIR         = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors'
IMG_FMT         = '{:06d}_color_0.jpg'

OUT_DIR         = './wrist_palm_dir_vis_auto'
FRAMES_VIS      = [147, 831, 234, 671, 358, 1155, 1647]

# 카메라 intrinsics
fx, fy = 605.9278, 606.1917
cx, cy = 414.0560, 260.2649

# ===== 카메라 Extrinsics (camera ← world) : 이전 공동최적화 결과 =====
R_cw = np.array([
    [ 0.22643170, -0.96805671, -0.10767957],
    [-0.36084943,  0.01931217, -0.93242411],
    [ 0.90471895,  0.24998648, -0.34494983]
], dtype=float)
t_cw = np.array([-0.01571018, 0.35198288, 0.29489303], dtype=float)

# 손목 오프셋 (손목 로컬 좌표계) — 공동최적화 결과
dL = np.array([-0.01958220,  0.03467778, -0.00843096], dtype=float)
dR = np.array([ 0.00669406, -0.00187255,  0.03843392], dtype=float)

# 손끝 장착 보정(현재 0)
dRvec_L = np.zeros(3); dT_L = np.zeros(3)
dRvec_R = np.zeros(3); dT_R = np.zeros(3)

# 손바닥 중심 오프셋(핸드 base 프레임) — 필요시 조정
pPalm_L = np.array([0.0, 0.0, 0.0], dtype=float)
pPalm_R = np.array([0.0, 0.0, 0.0], dtype=float)

# 로봇 바디 IMU (deg→rad)
body_imu = np.deg2rad([1.72, -15.86, -0.08])

# 손바닥 법선 방향(핸드 base 프레임). URDF 확인 후 반대면 [0,0,-1]로 바꾸고,
# 그래도 자동 부호 선택 로직이 있어 안정적으로 맞춰짐.
N_HAND_AXIS = np.array([0, 0, 1.0], dtype=float)

# 픽셀-미터 스케일 측정을 위한 작은 이동(m)
S_TEST = 0.01  # 1 cm
# 손목 픽셀에서 이동해 그릴 길이(px)
DPIX_L = 20.0
DPIX_R = 20.0

# 색상
COL_LW = (0,255,255)
COL_RW = (255,255,0)
COL_LT = (0,0,255)
COL_RT = (255,0,0)
COL_LP = (0,255,0)
COL_RP = (255,0,255)
COL_PX_DIR = (0,200,0)
COL_3D_EST = (0,128,0)

# =========================================================
# 유틸
# =========================================================
def rpy_to_mat(r, p, y):
    Rx = np.array([[1,0,0],[0,np.cos(r),-np.sin(r)],[0,np.sin(r),np.cos(r)]])
    Ry = np.array([[np.cos(p),0,np.sin(p)],[0,1,0],[-np.sin(p),0,np.cos(p)]])
    Rz = np.array([[np.cos(y),-np.sin(y),0],[np.sin(y),np.cos(y),0],[0,0,1]])
    return Rz @ Ry @ Rx

def rodrigues_to_R(rvec):
    theta = np.linalg.norm(rvec)
    if theta < 1e-12:
        return np.eye(3)
    k = rvec / theta
    Kc = np.array([[0, -k[2], k[1]],
                   [k[2], 0, -k[0]],
                   [-k[1], k[0], 0]])
    return np.eye(3) + np.sin(theta)*Kc + (1-np.cos(theta))*(Kc@Kc)

def project_point_world(p_world):
    """world 3D → pixel(+camera)"""
    p_cam = R_cw @ p_world + t_cw
    X, Y, Z = p_cam
    if Z <= 0:
        return None, p_cam
    u = fx * (X / Z) + cx
    v = fy * (Y / Z) + cy
    return (int(round(u)), int(round(v))), p_cam

# =========================================================
# 로드 및 맵
# =========================================================
robot = URDF.load(BODY_URDF_PATH)
left_hand = URDF.load(LEFT_HAND_URDF)
try:
    right_hand = URDF.load(RIGHT_HAND_URDF)
except Exception:
    right_hand = None

with open(DATA_PATH, 'r') as f:
    data = json.load(f)

l_arm_map = {
    'kLeftShoulderPitch':'left_shoulder_pitch_joint',
    'kLeftShoulderRoll' :'left_shoulder_roll_joint',
    'kLeftShoulderYaw'  :'left_shoulder_yaw_joint',
    'kLeftElbow'        :'left_elbow_joint',
    'kLeftWristRoll'    :'left_wrist_roll_joint',
    'kLeftWristPitch'   :'left_wrist_pitch_joint',
    'kLeftWristyaw'     :'left_wrist_yaw_joint',
}
r_arm_map = {
    'kRightShoulderPitch':'right_shoulder_pitch_joint',
    'kRightShoulderRoll' :'right_shoulder_roll_joint',
    'kRightShoulderYaw'  :'right_shoulder_yaw_joint',
    'kRightElbow'        :'right_elbow_joint',
    'kRightWristRoll'    :'right_wrist_roll_joint',
    'kRightWristPitch'   :'right_wrist_pitch_joint',
    'kRightWristYaw'     :'right_wrist_yaw_joint',
}

l_hand_map = {
    'kLeftHandPinky':'L_pinky_proximal_joint',
    'kLeftHandRing' :'L_ring_proximal_joint',
    'kLeftHandMiddle':'L_middle_proximal_joint',
    'kLeftHandIndex':'L_index_proximal_joint',
    'kLeftHandThumbBend':'L_thumb_proximal_pitch_joint',
    'kLeftHandThumbRotation':'L_thumb_proximal_yaw_joint',
}
r_hand_map = {
    'kRightHandPinky':'R_pinky_proximal_joint',
    'kRightHandRing' :'R_ring_proximal_joint',
    'kRightHandMiddle':'R_middle_proximal_joint',
    'kRightHandIndex':'R_index_proximal_joint',
    'kRightHandThumbBend':'R_thumb_proximal_pitch_joint',
    'kRightHandThumbRotation':'R_thumb_proximal_yaw_joint',
}

L_TIP_LINK = 'L_index_tip'
R_TIP_LINK = 'R_index_tip'

# =========================================================
# wrist→hand mount 추출
# =========================================================
def find_mount_transform(robot, wrist_link, prefer_side='left'):
    cand = []
    for j in robot.joints:
        if j.parent == wrist_link:
            child_low = j.child.lower()
            if ('hand' in child_low) or ('finger' in child_low):
                cand.append(j)
    if not cand:
        return None, None
    picked = cand[0]
    if len(cand) > 1:
        for j in cand:
            if prefer_side == 'left' and ('l_' in j.child.lower() or 'left' in j.child.lower()):
                picked = j; break
            if prefer_side == 'right' and ('r_' in j.child.lower() or 'right' in j.child.lower()):
                picked = j; break
    return picked, picked.origin

jL, T_wrist_to_hand_L = find_mount_transform(robot, 'left_wrist_yaw_link', 'left')
jR, T_wrist_to_hand_R = find_mount_transform(robot, 'right_wrist_yaw_link', 'right')
if T_wrist_to_hand_L is None: T_wrist_to_hand_L = np.eye(4)
if T_wrist_to_hand_R is None: T_wrist_to_hand_R = np.eye(4)

def safe_q(names_list, qpos_list, mapping):
    q = {}
    for n, v in zip(names_list, qpos_list):
        if n in mapping:
            q[mapping[n]] = v
    return q

def wrist_pose_world(frame, side='left'):
    if side == 'left':
        names = data['info']['joint_names']['left_arm']
        qpos  = data['data'][frame]['states']['left_arm']['qpos']
        amap  = l_arm_map
        wrist_link = 'left_wrist_yaw_link'
    else:
        names = data['info']['joint_names']['right_arm']
        qpos  = data['data'][frame]['states']['right_arm']['qpos']
        amap  = r_arm_map
        wrist_link = 'right_wrist_yaw_link'

    q = { amap[n]: v for n,v in zip(names, qpos) if n in amap }
    fk = robot.link_fk(q)
    T_world_base = np.eye(4)
    T_world_base[:3,:3] = rpy_to_mat(*body_imu)
    T_wrist = T_world_base @ fk[ robot.link_map[wrist_link] ]
    Rw = T_wrist[:3,:3]
    pw = T_wrist[:3,3]
    return Rw, pw

def left_index_tip_world(frame, d_off=dL, dRvec=dRvec_L, dT=dT_L):
    Rw, pw = wrist_pose_world(frame, 'left')
    T_off = np.eye(4); T_off[:3,3] = d_off
    R_delta = rodrigues_to_R(dRvec)
    T_mount = T_wrist_to_hand_L.copy()
    T_mount[:3,:3] = R_delta @ T_mount[:3,:3]
    T_mount[:3,3] += dT
    names = data['info']['joint_names']['left_hand']
    qpos  = data['data'][frame]['states']['left_hand']['qpos']
    qh = safe_q(names, qpos, l_hand_map)
    fk_h = left_hand.link_fk(qh)
    if L_TIP_LINK not in left_hand.link_map:
        return None
    T_hand_tip = fk_h[left_hand.link_map[L_TIP_LINK]]
    T_world_wrist = np.eye(4); T_world_wrist[:3,:3]=Rw; T_world_wrist[:3,3]=pw
    T_world_tip = T_world_wrist @ T_off @ T_mount @ T_hand_tip
    return T_world_tip[:3,3]

def right_index_tip_world(frame, d_off=dR, dRvec=dRvec_R, dT=dT_R):
    if (right_hand is None) or (R_TIP_LINK not in getattr(right_hand, 'link_map', {})):
        return None
    Rw, pw = wrist_pose_world(frame, 'right')
    T_off = np.eye(4); T_off[:3,3] = d_off
    R_delta = rodrigues_to_R(dRvec)
    T_mount = T_wrist_to_hand_R.copy()
    T_mount[:3,:3] = R_delta @ T_mount[:3,:3]
    T_mount[:3,3] += dT
    names = data['info']['joint_names']['right_hand']
    qpos  = data['data'][frame]['states']['right_hand']['qpos']
    qh = safe_q(names, qpos, r_hand_map)
    fk_h = right_hand.link_fk(qh)
    T_hand_tip = fk_h[right_hand.link_map[R_TIP_LINK]]
    T_world_wrist = np.eye(4); T_world_wrist[:3,:3]=Rw; T_world_wrist[:3,3]=pw
    T_world_tip = T_world_wrist @ T_off @ T_mount @ T_hand_tip
    return T_world_tip[:3,3]

def left_palm_world(frame, d_off=dL, palm_off=pPalm_L):
    Rw, pw = wrist_pose_world(frame, 'left')
    T_world_wrist = np.eye(4); T_world_wrist[:3,:3]=Rw; T_world_wrist[:3,3]=pw
    T_off = np.eye(4); T_off[:3,3] = d_off
    T_mount = T_wrist_to_hand_L
    T_palm = np.eye(4); T_palm[:3,3] = palm_off
    T_world_palm = T_world_wrist @ T_off @ T_mount @ T_palm
    return T_world_palm[:3,3]

def right_palm_world(frame, d_off=dR, palm_off=pPalm_R):
    if right_hand is None:
        return None
    Rw, pw = wrist_pose_world(frame, 'right')
    T_world_wrist = np.eye(4); T_world_wrist[:3,:3]=Rw; T_world_wrist[:3,3]=pw
    T_off = np.eye(4); T_off[:3,3] = d_off
    T_mount = T_wrist_to_hand_R
    T_palm = np.eye(4); T_palm[:3,3] = palm_off
    T_world_palm = T_world_wrist @ T_off @ T_mount @ T_palm
    return T_world_palm[:3,3]

# =========================================================
# 방향/부호 자동 선택 유틸
# =========================================================
def hand_mount_R(side):
    return T_wrist_to_hand_L[:3,:3] if side=='left' else T_wrist_to_hand_R[:3,:3]

def wrist_world_point(side, frame, use_offset=True):
    Rw, pw = wrist_pose_world(frame, side)
    if not use_offset:
        return pw
    return (Rw @ (dL if side=='left' else dR)) + pw

def palm_dir_world(side, frame, n_hand=N_HAND_AXIS):
    """월드에서의 palm 방향 단위벡터: Rw @ (R_mount @ n_hand)"""
    Rw, _ = wrist_pose_world(frame, side)
    Rm = hand_mount_R(side)
    v = Rw @ (Rm @ n_hand)
    n = np.linalg.norm(v)
    if n < 1e-12:
        return np.array([0,0,1.0], dtype=float)
    return v / n

def dirvec_uv_from_world(p_world, n_world, S=S_TEST):
    """월드 점에서 n_world 방향으로 S만큼 이동시 투영 → 이미지 방향 단위벡터와 스케일(px/m) 계산"""
    uv_w, _ = project_point_world(p_world)
    if uv_w is None:
        return None, None, None
    p2 = p_world + n_world * S
    uv_2, _ = project_point_world(p2)
    if uv_2 is None:
        return uv_w, None, None
    d = np.array([uv_2[0]-uv_w[0], uv_2[1]-uv_w[1]], dtype=float)
    n = np.linalg.norm(d)
    if n < 1e-12:
        return uv_w, None, None
    scale_per_m = n / S
    return np.array(uv_w, dtype=float), d / n, scale_per_m

def best_signed_dir(uv_w, d_uv_unit, uv_target, DPIX):
    """
    +d, -d 중 uv_target(예: URDF palm)과 더 가까운 쪽 선택.
    uv_w, d_uv_unit는 float 배열이어야 함.
    """
    if (uv_w is None) or (d_uv_unit is None):
        return +1.0, None, None, None
    cand_plus  = uv_w + DPIX * d_uv_unit
    cand_minus = uv_w - DPIX * d_uv_unit
    if uv_target is None:
        return +1.0, cand_plus.astype(int), np.linalg.norm(DPIX*d_uv_unit), None
    e_plus  = np.linalg.norm(cand_plus  - uv_target)
    e_minus = np.linalg.norm(cand_minus - uv_target)
    if e_minus < e_plus:
        return -1.0, cand_minus.astype(int), e_minus, e_plus
    else:
        return +1.0, cand_plus.astype(int), e_plus, e_minus

def auto_estimate_palm(side, frame, DPIX, uv_target=None, use_wrist_offset=True):
    """
    손목 기준 방향벡터로 픽셀/3D palm 추정.
    - uv_target 이 주어지면 부호 자동 선택.
    반환: (uv_px, uv_3d, p_3d, sign, scale_per_m)
    """
    p_ref = wrist_world_point(side, frame, use_offset=use_wrist_offset)
    n_world = palm_dir_world(side, frame)
    uv_w, d_unit, scale_per_m = dirvec_uv_from_world(p_ref, n_world, S=S_TEST)
    if (uv_w is None) or (d_unit is None) or (scale_per_m is None) or scale_per_m < 1e-9:
        return None, None, None, +1.0, None

    sign, uv_px, e_sel, e_other = best_signed_dir(uv_w, d_unit, (np.array(uv_target, dtype=float) if uv_target is not None else None), DPIX)

    # 3D 이동량(m) = (DPIX / scale_per_m) * sign
    s_m = (DPIX / scale_per_m) * sign
    p_3d = p_ref + s_m * n_world
    uv_3d, _ = project_point_world(p_3d)
    return (tuple(uv_px) if uv_px is not None else None), uv_3d, p_3d, sign, scale_per_m

# =========================================================
# 시각화
# =========================================================
def main():
    Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

    for fr in FRAMES_VIS:
        img_path = str(Path(IMG_DIR) / IMG_FMT.format(fr))
        img = cv2.imread(img_path)
        if img is None:
            print('[WARN] image not found:', img_path)
            continue
        vis = img.copy()

        # 손목 (오프셋 적용 위치)
        RwL, pwL = wrist_pose_world(fr, 'left');  p_wL = (RwL @ dL) + pwL
        RwR, pwR = wrist_pose_world(fr, 'right'); p_wR = (RwR @ dR) + pwR
        uv_wL, _ = project_point_world(p_wL)
        uv_wR, _ = project_point_world(p_wR)
        if uv_wL is not None:
            cv2.circle(vis, uv_wL, 8, COL_LW, -1); cv2.putText(vis, 'LW', (uv_wL[0]+6, uv_wL[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, COL_LW, 2)
        if uv_wR is not None:
            cv2.circle(vis, uv_wR, 8, COL_RW, -1); cv2.putText(vis, 'RW', (uv_wR[0]+6, uv_wR[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, COL_RW, 2)

        # URDF palm (hand base + pPalm)
        p_pL = left_palm_world(fr, palm_off=pPalm_L)
        p_pR = right_palm_world(fr, palm_off=pPalm_R)
        uv_pL, _ = project_point_world(p_pL) if p_pL is not None else (None, None)
        uv_pR, _ = project_point_world(p_pR) if p_pR is not None else (None, None)
        if uv_pL is not None:
            cv2.circle(vis, uv_pL, 7, COL_LP, -1); cv2.putText(vis, 'L_palm(URDF)', (uv_pL[0]+6, uv_pL[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.6, COL_LP, 2)
        if uv_pR is not None:
            cv2.circle(vis, uv_pR, 7, COL_RP, -1); cv2.putText(vis, 'R_palm(URDF)', (uv_pR[0]+6, uv_pR[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.6, COL_RP, 2)

        # 손끝(참고용)
        p_tL = left_index_tip_world(fr)
        p_tR = right_index_tip_world(fr)
        uv_tL, _ = project_point_world(p_tL) if p_tL is not None else (None, None)
        uv_tR, _ = project_point_world(p_tR) if p_tR is not None else (None, None)
        if uv_tL is not None:
            cv2.circle(vis, uv_tL, 6, COL_LT, -1); cv2.putText(vis, 'L_tip', (uv_tL[0]+6, uv_tL[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.6, COL_LT, 2)
        if uv_tR is not None:
            cv2.circle(vis, uv_tR, 6, COL_RT, -1); cv2.putText(vis, 'R_tip', (uv_tR[0]+6, uv_tR[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.6, COL_RT, 2)

        # ===== 자동 부호 선택 기반 추정 =====
        # LEFT
        uv_px_L, uv_3d_L, p3d_L, sign_L, scale_L = auto_estimate_palm('left',  fr, DPIX_L, uv_target=uv_pL, use_wrist_offset=True)
        if uv_wL is not None and uv_px_L is not None:
            cv2.arrowedLine(vis, uv_wL, uv_px_L, COL_PX_DIR, 2, tipLength=0.25)
            cv2.circle(vis, uv_px_L, 6, COL_PX_DIR, -1)
            txt = f'L_px{int(DPIX_L)} (sgn {int(sign_L)})'
            cv2.putText(vis, txt, (uv_px_L[0]+6, uv_px_L[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.55, COL_PX_DIR, 2)
        if uv_3d_L is not None:
            cv2.circle(vis, uv_3d_L, 6, COL_3D_EST, -1)
            cv2.putText(vis, 'L_palm(3D)', (uv_3d_L[0]+6, uv_3d_L[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.6, COL_3D_EST, 2)

        # RIGHT
        uv_px_R, uv_3d_R, p3d_R, sign_R, scale_R = auto_estimate_palm('right', fr, DPIX_R, uv_target=uv_pR, use_wrist_offset=True)
        if uv_wR is not None and uv_px_R is not None:
            cv2.arrowedLine(vis, uv_wR, uv_px_R, COL_PX_DIR, 2, tipLength=0.25)
            cv2.circle(vis, uv_px_R, 6, COL_PX_DIR, -1)
            txt = f'R_px{int(DPIX_R)} (sgn {int(sign_R)})'
            cv2.putText(vis, txt, (uv_px_R[0]+6, uv_px_R[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.55, COL_PX_DIR, 2)
        if uv_3d_R is not None:
            cv2.circle(vis, uv_3d_R, 6, COL_3D_EST, -1)
            cv2.putText(vis, 'R_palm(3D)', (uv_3d_R[0]+6, uv_3d_R[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.6, COL_3D_EST, 2)

        cv2.putText(vis, f'frame {fr}', (20,40), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0,255,0), 2)
        out_path = str(Path(OUT_DIR) / f'vis_{fr}.png')
        cv2.imwrite(out_path, vis)
        print('Saved:', out_path)

if __name__ == '__main__':
    main()


Saved: wrist_palm_dir_vis_auto/vis_147.png
Saved: wrist_palm_dir_vis_auto/vis_831.png
Saved: wrist_palm_dir_vis_auto/vis_234.png
Saved: wrist_palm_dir_vis_auto/vis_671.png
Saved: wrist_palm_dir_vis_auto/vis_358.png
Saved: wrist_palm_dir_vis_auto/vis_1155.png
Saved: wrist_palm_dir_vis_auto/vis_1647.png
